In [2]:
import pandas as pd
import numpy as np

populationCsvToArray = pd.read_csv("/Users/yassine/Desktop/data-python/fr_population.csv", sep=',')
populationBydate = populationCsvToArray.loc[populationCsvToArray['Année'] == 2013]
filtered_df = populationBydate[(populationBydate['Code zone'] != 96) & (populationBydate['Code zone'] != 41) & (populationBydate['Code zone'] != 128) & (populationBydate['Code zone'] != 214)]
populationRemoveDuplicateByCountries =  filtered_df.drop_duplicates(subset='Code zone', keep="last")
populationRemoveDuplicateByCountries.rename(columns={'Zone':'Pays'}, inplace=True)
dropPop = populationRemoveDuplicateByCountries.drop(['Élément','Code Produit','Symbole','Domaine','Code Domaine','Code Élément','Code année','Description du Symbole'], axis=1)
dropPop['Population'] = dropPop['Valeur'] * 1000
countries = dropPop
countries

,Code zone,Pays,Produit,Année,Unité,Valeur,Population
0,2,Afghanistan,Population,2013,1000 personnes,30552,30552000
1,202,Afrique du Sud,Population,2013,1000 personnes,52776,52776000
2,3,Albanie,Population,2013,1000 personnes,3173,3173000
3,4,Algérie,Population,2013,1000 personnes,39208,39208000
4,79,Allemagne,Population,2013,1000 personnes,82727,82727000
...,...,...,...,...,...,...,...
170,236,Venezuela (République bolivarienne du),Population,2013,1000 personnes,30405,30405000
171,237,Viet Nam,Population,2013,1000 personnes,91680,91680000
172,249,Yémen,Population,2013,1000 personnes,24407,24407000
173,251,Zambie,Population,2013,1000 personnes,14539,14539000


In [9]:
animauxCsv = pd.read_csv("/Users/yassine/Desktop/data-python/fr_animaux.csv", sep=',')
animauxCsv.loc[animauxCsv['Année'] == 2013]
animaux = animauxCsv.drop(['Année','Code Produit','Symbole','Domaine','Code Domaine','Code Élément','Code année','Description du Symbole'], axis=1)
animaux.rename(columns={'Valeur':'proteine'}, inplace=True)
animaux.rename(columns={'Zone':'Pays'}, inplace=True)

animauxGrammesperD = animaux[animaux['Élément'] == 'Disponibilité de protéines en quantité (g/personne/jour)']
animauxCaloriesPerD = animaux[animaux['Élément'] == 'Disponibilité alimentaire (Kcal/personne/jour)']

dataframeQuantities = animauxGrammesperD.merge(animauxCaloriesPerD, on='Code zone',suffixes=('_g/p/j', '_kcal/p/j'))

animauxPopulation = dataframeQuantities.merge(countries, on='Code zone')
animauxPopulation['food_supply_kg'] = animauxPopulation["proteine_g/p/j"] * 0.001 * animauxPopulation["Population"] * 365  
animauxPopulation["food_supply_kcal"] = animauxPopulation["proteine_kcal/p/j"] * animauxPopulation["Population"] * 365
animauxPopulation["ratio_kcal/kg"] =  animauxPopulation["food_supply_kcal"] / animauxPopulation['food_supply_kg']
animauxPopulation.dropna()

result = animauxPopulation.loc[animauxPopulation['proteine_g/p/j'] > 0.1]
clearData = result.loc[result['ratio_kcal/kg'] > 0]
clearData

,Code zone,Pays_g/p/j,Élément_g/p/j,Produit_g/p/j,Unité_g/p/j,proteine_g/p/j,Pays_kcal/p/j,Élément_kcal/p/j,Produit_kcal/p/j,Unité_kcal/p/j,proteine_kcal/p/j,Pays,Produit,Année,Unité,Valeur,Population,food_supply_kg,food_supply_kcal,ratio_kcal/kg
0,2,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Viande de Bovins,g/personne/jour,1.89,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Viande de Bovins,Kcal/personne/jour,27.0,Afghanistan,Population,2013,1000 personnes,30552,30552000,21076297.2,3.010900e+11,14285.714286
1,2,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Viande de Bovins,g/personne/jour,1.89,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Viande d'Ovins/Caprins,Kcal/personne/jour,31.0,Afghanistan,Population,2013,1000 personnes,30552,30552000,21076297.2,3.456959e+11,16402.116402
2,2,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Viande de Bovins,g/personne/jour,1.89,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Viande de Volailles,Kcal/personne/jour,9.0,Afghanistan,Population,2013,1000 personnes,30552,30552000,21076297.2,1.003633e+11,4761.904762
3,2,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Viande de Bovins,g/personne/jour,1.89,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),"Viande, Autre",Kcal/personne/jour,1.0,Afghanistan,Population,2013,1000 personnes,30552,30552000,21076297.2,1.115148e+10,529.100529
4,2,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Viande de Bovins,g/personne/jour,1.89,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Abats Comestible,Kcal/personne/jour,5.0,Afghanistan,Population,2013,1000 personnes,30552,30552000,21076297.2,5.575740e+10,2645.502646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76998,181,Zimbabwe,Disponibilité de protéines en quantité (g/pers...,Poissons Pelagiques,g/personne/jour,0.19,Zimbabwe,Disponibilité alimentaire (Kcal/personne/jour),Graisses Animales Crue,Kcal/personne/jour,25.0,Zimbabwe,Population,2013,1000 personnes,14150,14150000,981302.5,1.291188e+11,131578.947368
77001,181,Zimbabwe,Disponibilité de protéines en quantité (g/pers...,Poissons Pelagiques,g/personne/jour,0.19,Zimbabwe,Disponibilité alimentaire (Kcal/personne/jour),Oeufs,Kcal/personne/jour,6.0,Zimbabwe,Population,2013,1000 personnes,14150,14150000,981302.5,3.098850e+10,31578.947368
77002,181,Zimbabwe,Disponibilité de protéines en quantité (g/pers...,Poissons Pelagiques,g/personne/jour,0.19,Zimbabwe,Disponibilité alimentaire (Kcal/personne/jour),Lait - Excl Beurre,Kcal/personne/jour,49.0,Zimbabwe,Population,2013,1000 personnes,14150,14150000,981302.5,2.530728e+11,257894.736842
77003,181,Zimbabwe,Disponibilité de protéines en quantité (g/pers...,Poissons Pelagiques,g/personne/jour,0.19,Zimbabwe,Disponibilité alimentaire (Kcal/personne/jour),Poissons Eau Douce,Kcal/personne/jour,4.0,Zimbabwe,Population,2013,1000 personnes,14150,14150000,981302.5,2.065900e+10,21052.631579
